In [1]:
#######################################
#  Pandas Challenge - HeroesOfPymoli  #
#######################################

In [2]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

In [3]:
purchase_data.isnull().values.any()
purchase_data.columns

Index(['Purchase ID', 'SN', 'Age', 'Gender', 'Item ID', 'Item Name', 'Price'], dtype='object')

In [4]:
unique_data = purchase_data.drop_duplicates(subset = ["SN"])
df=pd.DataFrame([len(unique_data)], columns=["Total Players"]) 
df


,Total Players
0,576


In [5]:
#Purchase Summaries 

unique_items = purchase_data["Item ID"].unique()
number_unique_items = len(unique_items)
average_price = purchase_data["Price"].mean()
total_purchases = purchase_data["Purchase ID"].count()
total_revenue = purchase_data["Price"].sum()

summary_list = [{"Number of Unique Items": number_unique_items,
                "Average Price": average_price,
               "Total Number of Purchases": total_purchases,
               "Total Revenue": total_revenue}]
summary_df = pd.DataFrame(summary_list)
summary_df = summary_df.round(2)
summary_df["Average Price"]=summary_df["Average Price"].map("${0:,.2f}".format)
summary_df["Total Revenue"]=summary_df["Total Revenue"].map("${0:,.2f}".format)
summary_df

,Number of Unique Items,Average Price,Total Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [6]:
#Gender Demographics

unique_data = purchase_data.drop_duplicates(subset = ["SN"])

unique_player = unique_data[["Gender"]]
unique_gender_total = unique_player["Gender"].count()
unique_gender_total
male_total = unique_player["Gender"].value_counts()['Male']
male_percent = male_total/unique_gender_total*100
female_total = unique_player["Gender"].value_counts()['Female']
female_percent = female_total/unique_gender_total*100
other_total = unique_player["Gender"].value_counts()['Other / Non-Disclosed']
other_percent = other_total/unique_gender_total*100

gender_data = {"Total": [male_total, female_total, other_total], "Percent": [male_percent, female_percent, other_percent]} 
gender_df = pd.DataFrame(gender_data, index=["Male", "Female", "Other / Non-Disclosed"]) 
gender_df["Percent"]=gender_df["Percent"].map("{:.2f}%".format)
gender_df


,Total,Percent
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [7]:
#Puchase Analaysis by Gender

avg_price_group = purchase_data.groupby(["Gender"], as_index=False) ["Price"].mean()
avg_price=avg_price_group.rename(columns={"Price": "Average Price"})
avg_price=avg_price.sort_values(by=["Average Price"], ascending = False)

In [8]:
purchases_sum = purchase_data.groupby(["Gender"], as_index=False) ["Price"].sum()
purchases_sum_new = purchases_sum.rename(columns={"Price": "Purchases (Sum)"})
purchases_sum_new=purchases_sum_new


In [9]:
total_purchases_sum = purchases_sum_new["Purchases (Sum)"].sum(axis=0)


In [10]:
purchase_count = purchase_data.groupby(["Gender"], as_index=False) ["Purchase ID"].count()
purchase_count_new = purchase_count.rename(columns={"Purchase ID": "Purchases (Count)"})
purchase_count_new

,Gender,Purchases (Count)
0,Female,113
1,Male,652
2,Other / Non-Disclosed,15


In [11]:
average_purchase = purchase_data.groupby(["Gender"], as_index=False) ["Price"].sum()
average_purchase

,Gender,Price
0,Female,361.94
1,Male,1967.64
2,Other / Non-Disclosed,50.19


In [12]:
female_avg_purchase = average_purchase.iloc[0,1]/gender_df.iloc[1,0]
male_avg_purchase = average_purchase.iloc[1,1]/gender_df.iloc[0,0]
other_avg_purchase = average_purchase.iloc[2,1]/gender_df.iloc[2,0]
genders = {"Gender": ['Female', 'Male', 'Other / Non-Disclosed'], "Average Purchase Per Person": [female_avg_purchase, male_avg_purchase, other_avg_purchase]}
gender1_df = pd.DataFrame(genders, columns = ["Gender", "Average Purchase Per Person"])
gender1_df

,Gender,Average Purchase Per Person
0,Female,4.468395
1,Male,4.065372
2,Other / Non-Disclosed,4.562727


In [13]:
merged_df = pd.merge(purchase_count_new, avg_price, how="left")
merged_df = pd.merge(merged_df, purchases_sum_new)
merged_df = pd.merge(merged_df, gender1_df)
merged_df
merged_df ["Average Purchase Per Person"] = merged_df["Average Purchase Per Person"].map("${:.2f}".format)
merged_df ["Average Price"]= merged_df["Average Price"].map("${:,.2f}".format)
merged_df ["Purchases (Sum)"]= merged_df["Purchases (Sum)"].map("${:,.2f}".format)


In [14]:
sorted_merged_df = merged_df.sort_values(by=["Purchases (Count)"], ascending = False)
sorted_merged_df


,Gender,Purchases (Count),Average Price,Purchases (Sum),Average Purchase Per Person
1,Male,652,$3.02,"$1,967.64",$4.07
0,Female,113,$3.20,$361.94,$4.47
2,Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [15]:
#Age Demographics

unique_data["Age"].min()
unique_data["Age"].max()
bins=[0, 9, 14, 19, 24, 29, 34, 39, 45]
group_names=["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

In [16]:
pd.set_option('mode.chained_assignment', None)

In [17]:
unique_data["Age Bin"] = pd.cut(unique_data.Age, bins, labels=group_names, include_lowest=True)

In [18]:
age_count = unique_data.groupby(["Age Bin"], as_index=False) ["Purchase ID"].count() 
age_count=age_count.rename(columns={"Purchase ID": "Total Count"})
age_count["Percent"] = (age_count["Total Count"]/len(unique_data))*100
age_count["Percent"]= age_count["Percent"].map("{:.2f}%".format)
age_count

,Age Bin,Total Count,Percent
0,<10,17,2.95%
1,10-14,22,3.82%
2,15-19,107,18.58%
3,20-24,258,44.79%
4,25-29,77,13.37%
5,30-34,52,9.03%
6,35-39,31,5.38%
7,40+,12,2.08%


In [19]:
#Purchasing Analysis by Age

bins=[0, 9, 14, 19, 24, 29, 34, 39, 45]
group_names=["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

In [20]:
purchase_data["Age Bin"] = pd.cut(purchase_data.Age, bins, labels=group_names, include_lowest=True)


In [21]:
purchase_count = purchase_data.groupby(["Age Bin"], as_index=False) ["Purchase ID"].count()
purchase_count = purchase_count.rename(columns={"Purchase ID": "Purchase Count"})

In [22]:
purchase_count2 = purchase_data.groupby(["Age Bin"], as_index=False) ['Price'].sum()
purchase_count2 = purchase_count2.rename(columns={"Price": "Total Purchase Value"})

In [23]:
purchase_count3 = purchase_data.groupby(["Age Bin"], as_index=False) ['Price'].mean()
purchase_count3 = purchase_count3.rename(columns={"Price": "Avg Total Purchase Per Person"})

In [24]:
merged_purchases = pd.merge(purchase_count, purchase_count2, how="left")
merged_purchases = pd.merge(merged_purchases, purchase_count3, how="left")
merged_purchases["Average Purchase Price"]=merged_purchases["Total Purchase Value"]/merged_purchases["Purchase Count"]

In [25]:
merged_purchases["Average Purchase Price"]= merged_purchases["Average Purchase Price"].map("{:.2f}%".format)
merged_purchases["Total Purchase Value"]=merged_purchases["Total Purchase Value"].map("${:,.2f}".format)
merged_purchases["Avg Total Purchase Per Person"]= merged_purchases["Avg Total Purchase Per Person"].map("{:.2f}%".format)
merged_purchases

,Age Bin,Purchase Count,Total Purchase Value,Avg Total Purchase Per Person,Average Purchase Price
0,<10,23,$77.13,3.35%,3.35%
1,10-14,28,$82.78,2.96%,2.96%
2,15-19,136,$412.89,3.04%,3.04%
3,20-24,365,"$1,114.06",3.05%,3.05%
4,25-29,101,$293.00,2.90%,2.90%
5,30-34,73,$214.00,2.93%,2.93%
6,35-39,41,$147.67,3.60%,3.60%
7,40+,13,$38.24,2.94%,2.94%


* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [26]:
#Top Spenders
purchase_data.columns

Index(['Purchase ID', 'SN', 'Age', 'Gender', 'Item ID', 'Item Name', 'Price',
       'Age Bin'],
      dtype='object')

In [27]:
top_spenders = purchase_data.groupby(["SN"], as_index=False) ["Item ID"].count()
top_spenders = top_spenders.rename(columns={"Item ID": "Purchase Count"})
top_spenders = top_spenders.sort_values(by=['Purchase Count'], ascending=False)
top_spenders

,SN,Purchase Count
360,Lisosia93,5
275,Iral74,4
246,Idastidru52,4
75,Asur53,3
274,Inguron55,3
...,...,...
207,Hala31,1
206,Haisurra41,1
203,Hailaphos89,1
202,Haestyphos66,1


In [28]:
top_spend_sum = purchase_data.groupby(["SN"], as_index=False) ["Price"].sum()
top_spend_sum = top_spend_sum.rename(columns={"Price": "Total Purchase Value"})

In [29]:
top_spend_avg = purchase_data.groupby(["SN"], as_index=False) ["Price"].mean()
top_spend_avg = top_spend_avg.rename(columns={"Price": "Average Purchase Price"})
top_spend_avg = top_spend_avg.sort_values(by=["Average Purchase Price"], ascending = False)
top_spend_avg

,SN,Average Purchase Price
131,Dyally87,4.99
327,Lirtilsa71,4.94
564,Yarithsurgue62,4.94
450,Ririp86,4.94
119,Chanirrasta87,4.94
...,...,...
154,Eudanu84,1.02
426,Qilalista41,1.02
32,Aidai61,1.01
117,Chanirra79,1.01


In [30]:
merged_top_spend = pd.merge(top_spenders, top_spend_avg, how="left")
merged_top_spend = pd.merge(merged_top_spend, top_spend_sum, how="left")
merged_top_spend = merged_top_spend.sort_values(by=["Total Purchase Value"], ascending = False)
merged_top_spend

,SN,Purchase Count,Average Purchase Price,Total Purchase Value
0,Lisosia93,5,3.792000,18.96
2,Idastidru52,4,3.862500,15.45
26,Chamjask73,3,4.610000,13.83
1,Iral74,4,3.405000,13.62
32,Iskadarya95,3,4.366667,13.10
...,...,...,...,...
524,Frichjaskan98,1,1.020000,1.02
487,Isurria36,1,1.020000,1.02
341,Chanirra79,1,1.010000,1.01
400,Aidai61,1,1.010000,1.01


In [31]:
merged_top_spend = merged_top_spend.sort_values(by=["Total Purchase Value"], ascending=False)
merged_top_spend.columns

Index(['SN', 'Purchase Count', 'Average Purchase Price',
       'Total Purchase Value'],
      dtype='object')

In [32]:
#merged_top_spend["Average Purchase Price"]= merged_top_spend["Average Purchase Price"].map("${:,.2f}".format)
#merged_top_spend["Total Purchase Value"]= merged_top_spend["Total Purchase Value"].map("${:,.2f}".format)
merged_top_spend.set_index(["SN"])
merged_top_spend

,SN,Purchase Count,Average Purchase Price,Total Purchase Value
0,Lisosia93,5,3.792000,18.96
2,Idastidru52,4,3.862500,15.45
26,Chamjask73,3,4.610000,13.83
1,Iral74,4,3.405000,13.62
32,Iskadarya95,3,4.366667,13.10
...,...,...,...,...
329,Qilalista41,1,1.020000,1.02
571,Hala31,1,1.020000,1.02
341,Chanirra79,1,1.010000,1.01
400,Aidai61,1,1.010000,1.01


In [33]:
purchase_data

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price,Age Bin
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53,20-24
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56,40+
2,2,Ithergue48,24,Male,92,Final Critic,4.88,20-24
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27,20-24
4,4,Iskosia90,23,Male,131,Fury,1.44,20-24
...,...,...,...,...,...,...,...,...
775,775,Aethedru70,21,Female,60,Wolf,3.54,20-24
776,776,Iral74,21,Male,164,Exiled Doomblade,1.63,20-24
777,777,Yathecal72,20,Male,67,"Celeste, Incarnation of the Corrupted",3.46,20-24
778,778,Sisur91,7,Male,92,Final Critic,4.19,<10


In [47]:
#Most Popular Items

most_popular = purchase_data.groupby(["Item ID", "Item Name"], as_index=False) ["Purchase ID"].count()
most_popular = most_popular.rename(columns={"Purchase ID": "Purchase Count"})
most_popular2 = purchase_data.groupby(["Item ID"], as_index=False) ["Price"].sum()

merged_popular = pd.merge(most_popular, most_popular2, how="left")
popular_total_value = merged_popular["Purchase Count"] * merged_popular["Price"]

merged_popular["Total Purchase Value"]=popular_total_value
merged_popular["Total Purchase Value"]= merged_popular["Total Purchase Value"].map("${:,.2f}".format)
merged_popular["Price"]= merged_popular["Price"].map("${:,.2f}".format)
merged_popular.set_index(["Item ID"])
merged_popular.sort_values(by=["Purchase Count"], ascending=False)


,Item ID,Item Name,Purchase Count,Price,Total Purchase Value
90,92,Final Critic,13,$59.99,$779.87
174,178,"Oathbreaker, Last Hope of the Breaking Storm",12,$50.76,$609.12
141,145,Fiery Glass Crusader,9,$41.22,$370.98
129,132,Persuasion,9,$28.99,$260.91
105,108,"Extraction, Quickblade Of Trembling Hands",9,$31.77,$285.93
...,...,...,...,...,...
40,42,The Decapitator,1,$1.75,$1.75
49,51,Endbringer,1,$4.66,$4.66
115,118,"Ghost Reaver, Longsword of Magic",1,$2.17,$2.17
101,104,Gladiator's Glaive,1,$1.93,$1.93


In [48]:
#Most Profitable
profitable = purchase_data.groupby(["Item ID", "Item Name"], as_index=False) ["Purchase ID"].count()
profitable = profitable.rename(columns={"Purchase ID": "Purchase Count"})
profitable2 = purchase_data.groupby(["Item ID"], as_index=False) ["Price"].sum()

merged_profitable = pd.merge(profitable, profitable2, how="left")
profit = merged_profitable["Purchase Count"] * merged_profitable["Price"]
merged_profitable["Total Purchase Value"]=profit
most_profitable=merged_profitable.sort_values(by=["Total Purchase Value"], ascending=False)

most_profitable["Total Purchase Value"]= most_profitable["Total Purchase Value"].map("${:,.2f}".format)
most_profitable["Price"]= most_profitable["Price"].map("${:,.2f}".format)
merged_profitable.set_index(["Item ID"])
most_profitable


,Item ID,Item Name,Purchase Count,Price,Total Purchase Value
90,92,Final Critic,13,$59.99,$779.87
174,178,"Oathbreaker, Last Hope of the Breaking Storm",12,$50.76,$609.12
80,82,Nirvana,9,$44.10,$396.90
141,145,Fiery Glass Crusader,9,$41.22,$370.98
105,108,"Extraction, Quickblade Of Trembling Hands",9,$31.77,$285.93
...,...,...,...,...,...
88,90,Betrayer,1,$2.94,$2.94
115,118,"Ghost Reaver, Longsword of Magic",1,$2.17,$2.17
123,126,Exiled Mithril Longsword,1,$2.00,$2.00
101,104,Gladiator's Glaive,1,$1.93,$1.93
